<a href="https://colab.research.google.com/github/Razzf/MachineLearningTests/blob/master/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import os
import cv2
import random
from IPython.display import clear_output
from google.colab.patches import cv2_imshow

from sklearn import model_selection

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential



In [9]:
DATADIR = "/content/drive/My Drive/color"
CATEGORIES = ["Apple___Apple_scab", "Apple___Black_rot", "Apple___Cedar_apple_rust",
              "Apple___healthy", "Blueberry___healthy", "Cherry_(including_sour)___Powdery_mildew",
              "Cherry_(including_sour)___healthy", "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot",
              "Corn_(maize)___Common_rust_", "Corn_(maize)___Northern_Leaf_Blight", "Corn_(maize)___healthy",
              "Grape___Black_rot", "Grape___Esca_(Black_Measles)", "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
              "Grape___healthy", "Orange___Haunglongbing_(Citrus_greening)", "Peach___Bacterial_spot",
              "Peach___healthy", "Pepper,_bell___Bacterial_spot", "Pepper,_bell___healthy", "Potato___Early_blight",
              "Potato___Late_blight", "Potato___healthy", "Raspberry___healthy", "Soybean___healthy",
              "Squash___Powdery_mildew", "Strawberry___Leaf_scorch", "Strawberry___healthy", 
              "Tomato___Bacterial_spot", "Tomato___Early_blight", "Tomato___Late_blight", "Tomato___Leaf_Mold",
              "Tomato___Septoria_leaf_spot", "Tomato___Spider_mites Two-spotted_spider_mite",
              "Tomato___Target_Spot", "Tomato___Tomato_Yellow_Leaf_Curl_Virus", "Tomato___Tomato_mosaic_virus",
              "Tomato___healthy"]

In [10]:
training_data = []
counter = 0
#CATEGORIES = np.array(CATEGORIES)
CATEGORIES = CATEGORIES[28:]
print(CATEGORIES)

['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [11]:
IMG_SIZE = 224
def create_training_data():
  for category in CATEGORIES:
    counter = 0
    path = os.path.join(DATADIR, category)
    class_index = CATEGORIES.index(category)
    class_array = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for img in os.listdir(path):
      class_array = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
      counter = counter + 1
      print(class_index, ":", category, ": image number :", counter)
      class_array[class_index] = 1
      img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_ANYCOLOR)
      img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      #optional resize (in this case isn't necesary)
      training_data.append((new_array, class_array))
      if counter % 3 == 0:
        clear_output()
      if counter > 1000:
        break
    





In [12]:
create_training_data()

0 : Tomato___Bacterial_spot : image number : 94


KeyboardInterrupt: ignored

In [ ]:


random.shuffle(training_data)

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)



X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

In [ ]:
print(X[0])
print(y[0])

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.2)

model.fit(X, y, batch_size=16, epochs=25, validation_split=.2)

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.2)

from tensorflow.keras.models import Model
from tensorflow.keras import backend as K 
from tensorflow.keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten

import math 
from keras.optimizers import SGD 

def inception_module(x, fs_1x1, fs_3x3, fs_5x5, fs_pool_proj, fs_3x3_rd, fs_5x5_rd, name=None):
    
    conv_1x1 = Conv2D(fs_1x1, (1, 1), strides=(1, 1), padding='same', activation='relu')(x)
    
    conv_3x3 = Conv2D(fs_3x3_rd, (1, 1), strides=(1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(fs_3x3, (3, 3), strides=(1, 1), padding='same', activation='relu')(conv_3x3)

    conv_5x5 = Conv2D(fs_5x5_rd, (1, 1), strides=(1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(fs_5x5, (5, 5), strides=(1, 1), padding='same', activation='relu')(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(fs_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output


input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x, fs_1x1=64, fs_3x3_rd=96, fs_3x3=128, fs_5x5_rd=16, fs_5x5=32, fs_pool_proj=32, name='inception_3a')

x = inception_module(x,fs_1x1=128, fs_3x3_rd=128, fs_3x3=192, fs_5x5_rd=32, fs_5x5=96, fs_pool_proj=64, name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x, fs_1x1=192, fs_3x3_rd=96, fs_3x3=208, fs_5x5_rd=16, fs_5x5=48, fs_pool_proj=64, name='inception_4a')

x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x, fs_1x1=160, fs_3x3_rd=112, fs_3x3=224, fs_5x5_rd=24, fs_5x5=64, fs_pool_proj=64, name='inception_4b')

x = inception_module(x, fs_1x1=128, fs_3x3_rd=128, fs_3x3=256, fs_5x5_rd=24, fs_5x5=64, fs_pool_proj=64, name='inception_4c')

x = inception_module(x, fs_1x1=112, fs_3x3_rd=144, fs_3x3=288, fs_5x5_rd=32, fs_5x5=64, fs_pool_proj=64, name='inception_4d')

x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x, fs_1x1=256, fs_3x3_rd=160, fs_3x3=320, fs_5x5_rd=32, fs_5x5=128, fs_pool_proj=128, name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x, fs_1x1=256, fs_3x3_rd=160, fs_3x3=320, fs_5x5_rd=32, fs_5x5=128, fs_pool_proj=128, name='inception_5a')

x = inception_module(x, fs_1x1=384, fs_3x3_rd=192, fs_3x3=384, fs_5x5_rd=48, fs_5x5=128, fs_pool_proj=128, name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(10, activation='softmax', name='output')(x)



model = Model(input_layer, [x, x1, x2], name='inception_v1')

sgd = SGD(lr=0.01, momentum=0.9, nesterov=False)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, [y_train, y_train, y_train], validation_split=[0.2, 0.2, 0.2], epochs=25, batch_size=16)